In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import polars as pl
from utilsforecast.plotting import plot_series
from statsforecast import StatsForecast
from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate
import numpy as np
from plotly.subplots import make_subplots
from statsmodels.stats.diagnostic import acorr_ljungbox

import polars as pl
import plotly.express as px
from utilsforecast.plotting import plot_series
from utilsforecast.losses import *
from statsmodels.tsa.seasonal import STL, seasonal_decompose, MSTL
import numpy as np

import plotly.graph_objects as go
import plotly.io as pio
from tsfeatures import tsfeatures, stl_features

from utilsforecast.losses import *
from statsmodels.nonparametric.smoothers_lowess import lowess
from utilsforecast.feature_engineering import fourier, pipeline
from sklearn.linear_model import LinearRegression
from functools import partial
from plotting_utils import (
    plotly_series as plot_series,
    plot_decomposition,
    plot_residuals_diagnostic,
)
from summary_utils import get_fitted_residuals
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pl.read_parquet(
    "data/london_smart_meters/preprocessed/london_smart_meters_merged_block_0-7.parquet"
)
timestamp = data.group_by("LCLid").agg(
    pl.datetime_range(
        start=pl.col("start_timestamp"),
        end=pl.col("start_timestamp").dt.offset_by(
            pl.format("{}m", pl.col("series_length").sub(1).mul(30))
        ),
        interval="30m",
    ).alias("ds"),
)
data = timestamp.join(data, on="LCLid", how="inner").rename(
    {"LCLid": "unique_id", "energy_consumption": "y"}
)
data.head(5)

In [ ]:
id_ = "unique_id"
time_ = "ds"
target_ = "y"
id_col = pl.col(id_)
time_col = pl.col(time_)
target_col = pl.col(target_)

In [ ]:
data = (
    data.filter(pl.col("file").eq("block_7"))
    .select(
        [
            time_,
            id_,
            target_,
            "Acorn",
            "Acorn_grouped",
            "holidays",
            "visibility",
            "windBearing",
            "temperature",
            "dewPoint",
            "pressure",
            "apparentTemperature",
            "windSpeed",
            "precipType",
            "icon",
            "humidity",
            "summary",
        ]
    )
    .explode(
        [
            time_,
            target_,
            "holidays",
            "visibility",
            "windBearing",
            "temperature",
            "dewPoint",
            "pressure",
            "apparentTemperature",
            "windSpeed",
            "precipType",
            "icon",
            "humidity",
            "summary",
        ]
    )
)
data.head()

In [ ]:
selected_id = "MAC000193"
data = data.filter(pl.col(id_).eq(selected_id))
data.head()

## Baseline Forecasts in Time Series

Baseline forecasts are simple, interpretable models that serve as reference points for evaluating more complex forecasting methods. They help us understand whether sophisticated models truly add value, or if simple heuristics are sufficient for the problem at hand. In practice, a good forecasting workflow always starts with strong baselines.

Below are some common baseline models used in time series forecasting, along with their mathematical formulations:

### 1. Naive Forecast

The naive forecast assumes that the next value in the series will be the same as the last observed value.

$$
\hat{y}_{t+1} = y_t
$$

This approach is especially effective for random walk or highly persistent series.

---

### 2. Historic Average

The historic average predicts the next value as the mean of all observed values up to the current time.

$$
\hat{y}_{t+1} = \frac{1}{t} \sum_{i=1}^{t} y_i
$$

This method works well for stationary series without strong trends or seasonality.

---

### 3. Seasonal Naive

The seasonal naive forecast repeats the value from the same season in the previous cycle. For example, with half-hourly data and daily seasonality ($s=48$):

$$
\hat{y}_{t+1} = y_{t+1-s}
$$

where $s$ is the season length (here, $s=48$ for daily seasonality).

---

### 4. Drift Forecast

This model extends the naive forecast by adding a constant drift (trend) term, estimated from the historical average change.

$$
\hat{y}_{t+1} = y_t + \hat{d}
$$

where the drift $\hat{d}$ is typically estimated as:

$$
\hat{d} = \frac{y_t - y_1}{t-1}
$$

#### Derivation

Suppose we have a time series $\{y_1, y_2, \ldots, y_t\}$.

The random walk with drift model is:
$$
y_{k+1} = y_k + d
$$
where $d$ is the drift (constant increment), and $\varepsilon_{k+1}$ is a noise term.

If we recursively expand this from $y_1$:
$$
y_2 = y_1 + d \\
y_3 = y_2 + d = y_1 + 2d \\
\vdots \\
y_t = y_1 + (t-1)d + 
$$

This means
$$
d = \frac{y_t - y_1}{t-1}
$$

So the value at time $k$ is:
$$
y_k = y_1 + (k-1)\frac{y_t - y_1}{t-1}
$$

This is the equation of a straight line passing through $(1, y_1)$ and $(t, y_t)$, showing that the random walk with drift essentially fits a line between the first and last points of the series.

This means the forecast advances along a line defined by the starting and ending values, without considering any intermediate fluctuations or seasonality. It provides a simple way to capture a linear trend in the data.

Hence, it is useful for series with a linear trend.

---
By comparing advanced models to these baselines, we can better assess the value added by more complex approaches. If a sophisticated model does not outperform these simple baselines, it may not be worth the extra complexity.

In [ ]:
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    RandomWalkWithDrift,
)

fcst = StatsForecast(
    models=[
        Naive(),
        HistoricAverage(),
        SeasonalNaive(season_length=48, alias="DailySeasonalNaive"),
        SeasonalNaive(season_length=48 * 7, alias="WeeklySeasonalNaive"),
        RandomWalkWithDrift(),
    ],
    freq="30m",
)

In [ ]:
y_hat = fcst.cross_validation(
    df=data.select([id_, time_, target_col.forward_fill()]),
    fitted=True,
    h=48,
    n_windows=1,
    step_size=48,
).drop("cutoff")

In [ ]:
plot_series(data, y_hat, max_insample_length=200, width=1400)

Electricity load demand data typically exhibits strong seasonality with no trend. As a result, simple baseline methods such as the naive forecast, historic average, and drift are not well-suited for this type of data

In contrast, models that explicitly account for seasonality, such as the daily and weekly seasonal naive methods, perform significantly better. Among these, the weekly seasonal naive model tends to track the actual time series more closely, as it leverages the repeated weekly consumption patterns inherent in electricity demand data. This highlights the importance of incorporating seasonality into forecasting models for load demand.

### Model Validation Metrics

To evaluate and compare forecasting models, we use several standard error metrics. Each metric provides a different perspective on model performance:

#### 1. Mean Absolute Error (MAE)
Measures the average magnitude of errors in the forecasts, without considering their direction.

$$
\mathrm{MAE} = \frac{1}{n} \sum_{t=1}^n |y_t - \hat{y}_t|
$$

- **Interpretation:** Lower MAE indicates better accuracy. It is easy to interpret but does not penalize large errors more than small ones.

---

#### 2. Mean Squared Error (MSE)
Calculates the average of the squared differences between actual and predicted values.

$$
\mathrm{MSE} = \frac{1}{n} \sum_{t=1}^n (y_t - \hat{y}_t)^2
$$

- **Interpretation:** Penalizes larger errors more heavily. Useful when large errors are particularly undesirable.

---

#### 3. Root Mean Squared Error (RMSE)
The square root of MSE, bringing the error metric back to the original scale of the data.

$$
\mathrm{RMSE} = \sqrt{\mathrm{MSE}}
$$

- **Interpretation:** Like MSE, but easier to interpret since it is in the same units as the data.

---

#### 4. Mean Absolute Percentage Error (MAPE)
Expresses forecast error as a percentage of the actual values.

$$
\mathrm{MAPE} = \frac{100\%}{n} \sum_{t=1}^n \left| \frac{y_t - \hat{y}_t}{y_t} \right|
$$

- **Interpretation:** Useful for comparing forecast accuracy across series of different scales. Sensitive to zero or near-zero actual values.

---

#### 5. Symmetric Mean Absolute Percentage Error (sMAPE)
A variation of MAPE that treats over- and under-forecasts more symmetrically.

$$
\mathrm{sMAPE} = \frac{100\%}{n} \sum_{t=1}^n \frac{|y_t - \hat{y}_t|}{(|y_t| + |\hat{y}_t|)/2}
$$

- **Interpretation:** Bounded between 0% and 200%. Less sensitive to outliers and zero values than MAPE.

---

#### 6. Mean Absolute Scaled Error (MASE) with Seasonality 48
Compares the forecast error to the average error of a naive seasonal forecast (here, with a seasonality of 48, e.g., daily for half-hourly data).

$$
\mathrm{MASE} = \frac{\mathrm{MAE}}{\frac{1}{n-s} \sum_{t=s+1}^n |y_t - y_{t-s}|}
$$

- **Interpretation:** 
    - MASE < 1: Model outperforms the seasonal naive forecast.
    - MASE > 1: Model is worse than the seasonal naive.
    - Seasonality = 48 means the benchmark is the value from the same time on the previous day.

---

**Summary:**  
- Use MAE and RMSE for absolute error measurement.
- Use MAPE and sMAPE for relative error (percentage-based).
- Use MASE to benchmark against a simple seasonal naive model, especially for seasonal data.  
Combining these metrics gives a comprehensive view of model performance.

In [ ]:
from functools import partial

metrics = [
    mae,
    mse,
    rmse,
    mape,
    smape,
    partial(mase, seasonality=48),
]
evaluate(
    y_hat,
    metrics=metrics,
    train_df=data.select([id_, time_, target_]),
)

The evaluation metrics confirm the visual findings from the plots: the naive, mean (historic average), and drift models yield relatively high error values, indicating poor performance on this dataset. This is expected, as these models do not account for the strong seasonality present in electricity load demand.

In contrast, the weekly seasonal naive model achieves significantly lower errors across all metrics

In [ ]:
y_hat = fcst.forecast(
    df=data.select([id_, time_, target_col.forward_fill()]),
    h=48,
    fitted=True,
)

In [ ]:
fitted_residuals = get_fitted_residuals(fcst).drop_nans()

### Residual Analysis

Residual analysis is a crucial step in time series modeling, as it helps assess whether the chosen model adequately captures the underlying patterns in the data. By examining the residuals—the differences between observed values and model predictions—we can check for remaining structure, autocorrelation, or non-randomness.

Ideally, residuals should resemble white noise: they should be randomly distributed with constant variance and no discernible patterns over time. If residuals display autocorrelation, seasonality, or changing variance, this indicates that the model has not fully captured important aspects of the data, and further refinement or alternative modeling approaches may be necessary.

Statistical tests, such as the Ljung-Box test, can formally assess whether residuals are uncorrelated. Visual diagnostics, including residual plots and autocorrelation plots, provide additional insights into model adequacy and potential improvements.

In [ ]:
model = "WeeklySeasonalNaive"

residuals = fitted_residuals.get_column(model).drop_nulls().to_numpy()
time = fitted_residuals.get_column(time_).drop_nulls().to_numpy()

plot_residuals_diagnostic(time=time, residuals=residuals)

The residual plot reveals significant autocorrelation, indicating that the residuals are not white noise. This suggests that the model has not fully captured all the underlying patterns or dependencies in the data

### Ljung-Box test
The Ljung-Box test is a statistical test used to check whether any group of autocorrelations of a time series are significantly different from zero. In other words, it tests whether the residuals (errors) from a time series model are independently distributed (i.e., exhibit no autocorrelation).

### Purpose

- **Null hypothesis ($H_0$):** The data are independently distributed (no autocorrelation up to lag $h$).
- **Alternative hypothesis ($H_1$):** The data are not independently distributed (there is autocorrelation at one or more lags up to $h$).

### Test Statistic

The Ljung-Box test statistic is calculated as:

$$
Q = n(n+2) \sum_{k=1}^h \frac{\hat{\rho}_k^2}{n-k}
$$

where:

- $n$ = number of observations
- $h$ = number of lags being tested
- $\hat{\rho}_k$ = sample autocorrelation at lag $k$

### Distribution

- Under the null hypothesis, $Q$ approximately follows a chi-squared distribution with $h$ degrees of freedom:
    $$
    Q \sim \chi^2_h
    $$

### Steps

1. **Compute residuals** from your time series model.
2. **Calculate sample autocorrelations** $\hat{\rho}_k$ for lags $k = 1, 2, ..., h$.
3. **Compute $Q$** using the formula above.
4. **Compare $Q$** to the critical value from the chi-squared distribution with $h$ degrees of freedom, or compute the p-value.
5. **Decision:**  
     - If the p-value is small (e.g., $< 0.05$), reject $H_0$ (there is significant autocorrelation).
     - If the p-value is large, do not reject $H_0$ (no evidence of autocorrelation).

### Interpretation

- **Low p-value:** Residuals are autocorrelated; the model may be inadequate.
- **High p-value:** No evidence of autocorrelation; residuals resemble white noise.

### Summary Table

| Step | Description |
|------|-------------|
| 1    | Compute residuals from model |
| 2    | Calculate autocorrelations up to lag $h$ |
| 3    | Compute $Q$ statistic |
| 4    | Compare $Q$ to $\chi^2_h$ distribution |
| 5    | Interpret p-value |

The Ljung-Box test is widely used for model diagnostics in time series analysis to ensure that the model has captured all temporal dependencies.

In [ ]:
resid_test = acorr_ljungbox(residuals, boxpierce=True)
resid_test

The Ljung-Box test result shows a p-value of 0, which means we reject the null hypothesis of no autocorrelation in the residuals. This indicates that significant autocorrelation remains, which is inline with what we observe in the ACF plot above